# In this notebook we will create a graph from the already preprocessed timetable dataset 

In [1]:
import os
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import math
import networkx as nx
import matplotlib.pyplot as plt
from heapq import heappush, heappop
from datetime import datetime
import heapq
import copy
import numpy as np
from IPython.display import display, HTML
from ipywidgets import widgets, Output, HBox, VBox, Button, Tab, BoundedFloatText, Combobox
import plotly.graph_objects as go
import random

pd.set_option("display.max_columns", 50)

In [2]:
# Load the CSV file
df = pd.read_csv('data/lausanne_pairs_df.csv')
# change the route_desc column so that if it's B it becomes Bus and if it's WALKING it stays WALKING and otherwize becomes Zug
df['route_desc'] = df['route_desc'].apply(lambda x: 'Bus' if x == 'B' else 'Zug' )

In [3]:
# parse time function that strips the date and converts the time to seconds without using datetime 
def parse_time(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h *60 + m 

# reverse function that takes a time in minutes and returns a string
def reverse_time(time_int):
    h = time_int // 60
    m = time_int % 60
    return f'{h:02d}:{m:02d}:00'

# Apply cleaning to the dataframe
df['arrival_time_station1'] = df['arrival_time_station1'].apply(parse_time)
df['arrival_time_station2'] = df['arrival_time_station2'].apply(parse_time)
df = df.groupby(['station1_name', 'arrival_time_station1', 'station2_name', 'arrival_time_station2', 'route_id']).first().reset_index()
df=df[['station1_name', 'arrival_time_station1', 'station2_name', 'arrival_time_station2', 'route_id','route_desc','station1_lat','station1_lon','station2_lat','station2_lon']]
df

,station1_name,arrival_time_station1,station2_name,arrival_time_station2,route_id,route_desc,station1_lat,station1_lon,station2_lat,station2_lon
0,"Aclens, collège",328,"Romanel-sur-Morges, poste",330,92-735-j24-1,Bus,46.567231,6.510288,46.555761,6.510810
1,"Aclens, collège",356,"Romanel-sur-Morges,Z.I. Moulin",358,92-736-D-j24-1,Bus,46.567231,6.510288,46.562759,6.521841
2,"Aclens, collège",386,"Gollion, village",389,92-735-j24-1,Bus,46.567231,6.510288,46.586114,6.509318
3,"Aclens, collège",388,"Romanel-sur-Morges, poste",390,92-735-j24-1,Bus,46.567231,6.510288,46.555761,6.510810
4,"Aclens, collège",416,"Romanel-sur-Morges,Z.I. Moulin",418,92-736-D-j24-1,Bus,46.567231,6.510288,46.562759,6.521841
...,...,...,...,...,...,...,...,...,...,...
222971,"Vufflens-la-Ville,grande salle",1140,"Penthaz, village",1144,92-58-C-j24-1,Bus,46.581532,6.542017,46.599466,6.539008
222972,"Vufflens-la-Ville,grande salle",1166,"Vufflens-la-Ville, église",1168,92-58-C-j24-1,Bus,46.581532,6.542017,46.577006,6.539358
222973,"Vufflens-la-Ville,grande salle",1170,"Penthaz, village",1174,92-58-C-j24-1,Bus,46.581532,6.542017,46.599466,6.539008
222974,"Vufflens-la-Ville,grande salle",1226,"Vufflens-la-Ville, église",1228,92-58-C-j24-1,Bus,46.581532,6.542017,46.577006,6.539358


In [4]:
# read csv file
walking_df = pd.read_csv('data/walking_graph.csv')
# change walking time from seconds to minutes and floor it to the nearest minute
walking_df['walking_time'] = walking_df['walking_time'].apply(lambda x: math.ceil(x/60))
walking_df = walking_df.groupby(['station1_name', 'station2_name','walking_time','route']).first().reset_index()
walking_df = walking_df[['station1_name', 'station2_name','walking_time','route','route_desc','station1_lat','station1_lon','station2_lat','station2_lon']]
walking_df

,station1_name,station2_name,walking_time,route,route_desc,station1_lat,station1_lon,station2_lat,station2_lon
0,Bel-Air LEB,"Cheseaux-sur-L., Bel-Air",1,WALKING,WALKING,46.578680,6.605528,46.578482,6.605779
1,"Belmont-sur-L., Blessoney","Belmont-sur-L., centre",9,WALKING,WALKING,46.519513,6.683942,46.519606,6.678444
2,"Belmont-sur-L., Burenoz","Belmont-sur-L., Grands Champs",8,WALKING,WALKING,46.521194,6.672542,46.522412,6.677294
3,"Belmont-sur-L., Burenoz","Belmont-sur-L., Malavaux",6,WALKING,WALKING,46.521194,6.672542,46.520607,6.676360
4,"Belmont-sur-L., Burenoz","Belmont-sur-L., Pâquis",8,WALKING,WALKING,46.521194,6.672542,46.520224,6.677178
...,...,...,...,...,...,...,...,...,...
3277,"Vufflens-la-Ville, gare",Vufflens-la-Ville,2,WALKING,WALKING,46.577179,6.531345,46.576784,6.530662
3278,"Vufflens-la-Ville, gare",Vufflens-la-Ville,3,WALKING,WALKING,46.577179,6.531345,46.576247,6.530770
3279,"Vufflens-la-Ville, gare",Vufflens-la-Ville,6,WALKING,WALKING,46.577179,6.531345,46.574703,6.530519
3280,"Vufflens-la-Ville, gare","Vufflens-la-Ville, Cuvillard",8,WALKING,WALKING,46.577179,6.531345,46.579624,6.534938


In [5]:
# Create a MultiDiGraph
G = nx.MultiDiGraph()

# Add edges to the graph
for _, row in df.iterrows():
    station1 = row['station1_name']
    station2 = row['station2_name']
    departure_time = row['arrival_time_station1']
    arrival_time = row['arrival_time_station2']
    route_id = row['route_id']
    route_desc = row['route_desc']
    station1_lat = row['station1_lat']
    station1_lon = row['station1_lon']
    station2_lat = row['station2_lat']
    station2_lon = row['station2_lon']
    
    # Add edge with departure and arrival times as attributes
    G.add_edge(station1, station2, dep_time=departure_time, arr_time=arrival_time, route_id=route_id, route_desc=route_desc, station1_lat=station1_lat, station1_lon=station1_lon, station2_lat=station2_lat, station2_lon=station2_lon)


# Add edges to the graph from walking data
for _, row in walking_df.iterrows():
    station1 = row['station1_name']
    station2 = row['station2_name']
    walking_time = row['walking_time']
    route_id = row['route']
    route_desc = row['route_desc']
    station1_lat = row['station1_lat']
    station1_lon = row['station1_lon']
    station2_lat = row['station2_lat']
    station2_lon = row['station2_lon']
    
    # Add edge with walking time as attributes 
    G.add_edge(station1, station2, dep_time=0, arr_time=walking_time, route_id=route_id, route_desc=route_desc, station1_lat=station1_lat, station1_lon=station1_lon, station2_lat=station2_lat, station2_lon=station2_lon)

In [6]:
def latest_departure_time(G, start_station, end_station, arrival_time):
    # Initialize dictionaries to store latest departure times, paths, and route details
    latest_departure = {node: float('-inf') for node in G.nodes}
    latest_departure[end_station] = arrival_time
    path = {node: [] for node in G.nodes}
    path[end_station] = [end_station]
    path_details = {node: [] for node in G.nodes}

    # Priority queue for processing nodes
    queue = []
    heapq.heappush(queue, (-arrival_time, end_station))

    while queue:
        current_time, current_station = heapq.heappop(queue)
        current_time = -current_time

        for pred in G.predecessors(current_station):
            for key, edge in G[pred][current_station].items():
                dep_time = edge['dep_time']
                arr_time = edge['arr_time']
                route_id = edge['route_id']
                route_desc = edge['route_desc']
                from_lat = edge['station1_lat']
                from_lon = edge['station1_lon']
                to_lat = edge['station2_lat']
                to_lon = edge['station2_lon']

                # Calculate the potential latest departure time
                if dep_time == 0:  # Walking edge
                    potential_dep_time = current_time - arr_time
                    if potential_dep_time < 0:
                        continue
                else:  # Train edge
                    if arr_time <= current_time:
                        potential_dep_time = dep_time
                    else:
                        continue
                
                # Ensure at least a 2-minute difference for non-walking routes
                if path_details[current_station]:
                    last_route_id = path_details[current_station][-1]['route_id']
                    if last_route_id != "WALKING" and route_id != "WALKING" and last_route_id != route_id:
                        if current_time - arr_time < 2:
                            continue

                # Account for waiting time
                wait_time = max(0, current_time - arr_time)
                actual_arrival_time = current_time - wait_time

                if potential_dep_time > latest_departure[pred]:
                    latest_departure[pred] = potential_dep_time
                    path[pred] = path[current_station] + [pred]
                    path_details[pred] = path_details[current_station] + [{
                        'from': pred,
                        'to': current_station,
                        'departure': dep_time if dep_time != 0 else potential_dep_time,
                        'arrival': current_time if route_id == "WALKING" else actual_arrival_time,
                        'route_id': route_id,
                        'route_desc': route_desc,
                        'from_lat': from_lat,
                        'from_lon': from_lon,
                        'to_lat': to_lat,
                        'to_lon': to_lon
                    }]
                    heapq.heappush(queue, (-potential_dep_time, pred))

    # Reverse the path and path details to return them in the correct order
    reversed_path = list(reversed(path[start_station]))
    reversed_path_details = list(reversed(path_details[start_station]))

    return latest_departure[start_station], reversed_path, reversed_path_details

# Example usage
#start_station = "Lausanne-Flon, pl. de l'Europe"
#end_station = 'St-Sulpice VD, Venoge sud'
#arrival_time = parse_time("13:00:00")
#latest_departure, path, path_details = latest_departure_time(G, start_station, end_station, arrival_time)

# Finding the latest departure time and path
#if latest_departure == float('-inf'):
#    print(f"No valid path found from {start_station} to {end_station} arriving by {arrival_time} minutes after midnight.")
#else:
#    print(f"Latest departure time from {start_station}: {latest_departure} minutes after midnight")
#    print(f"Path: {' -> '.join(path)}")
#    for detail in path_details:
#        print(f"From: {detail['from']}, To: {detail['to']}, Departure: {reverse_time(detail['departure'])}, Arrival: {reverse_time(detail['arrival'])}, Route ID: {detail['route_id']}, Route Desc: {detail['route_desc']}")
#print(path_details)

# YEN

In [7]:
def delete_edge_with_attributes(graph, station1, station2, dep_time=None, arr_time=None, route_id=None, route_desc=None, station1_lat=None, station1_lon=None, station2_lat=None, station2_lon=None):
    for u, v, key, data in graph.edges(keys=True, data=True):
        if route_id == "WALKING":
            arr_time = arr_time - dep_time
            dep_time = 0

        if (u == station1 and v == station2):
            if (dep_time is None or data.get('dep_time') == dep_time) and \
               (arr_time is None or data.get('arr_time') == arr_time) and \
               (route_id is None or data.get('route_id') == route_id) and \
               (route_desc is None or data.get('route_desc') == route_desc) and \
               (station1_lat is None or data.get('station1_lat') == station1_lat) and \
               (station1_lon is None or data.get('station1_lon') == station1_lon) and \
               (station2_lat is None or data.get('station2_lat') == station2_lat) and \
               (station2_lon is None or data.get('station2_lon') == station2_lon):
                    graph.remove_edge(u, v, key=key)
                    #print(f"Deleted edge: {u} -> {v}")
                    return True
    #print(f"Edge not found: {station1} -> {station2}")
    return False

def add_edge_with_attributes(graph, station1, station2, dep_time=None, arr_time=None, route_id=None, route_desc=None, station1_lat=None, station1_lon=None, station2_lat=None, station2_lon=None):
    if route_id == "WALKING":
        arr_time = arr_time - dep_time
        dep_time = 0

    graph.add_edge(station1, station2, dep_time=dep_time, arr_time=arr_time, route_id=route_id, route_desc=route_desc, station1_lat=station1_lat, station1_lon=station1_lon, station2_lat=station2_lat, station2_lon=station2_lon)
    print(f"Added edge: {station1} -> {station2}")

# Function to find the top 5 trips
def find_top_k_trips(G, source, target, arrival_time, k=5):

    top_routes = []
    
    # Find the initial best route
    latest_departure, reversed_path, reversed_path_details = latest_departure_time(G, source, target, arrival_time)
    if latest_departure is not None:
        top_routes.append((latest_departure, reversed_path_details))

    all_edges_to_delete = reversed_path_details

    # Remove all collected edges
    for detail in all_edges_to_delete:
        dep_time = detail['departure']
        arr_time = detail['arrival']
        u = detail['from']
        v = detail['to']
        route_id = detail['route_id']
        route_desc = detail['route_desc']
        station1_lat = detail['from_lat']
        station1_lon = detail['from_lon']
        station2_lat = detail['to_lat']
        station2_lon = detail['to_lon']
        
        delete_edge_with_attributes(G, u, v, dep_time, arr_time, route_id,route_desc, station1_lat, station1_lon, station2_lat, station2_lon) 
        # Find the next best route after removing edges

        new_latest_departure, new_reversed_path, new_reversed_path_details = latest_departure_time(G, source, target, arrival_time)

        #check if new best route is not present in top routes
        if (new_latest_departure, new_reversed_path_details) not in top_routes:
            top_routes.append((new_latest_departure, new_reversed_path_details))

        # Add the removed edge back to the graph
        add_edge_with_attributes(G, u, v, dep_time, arr_time, route_id,route_desc, station1_lat, station1_lon, station2_lat, station2_lon)

    print(top_routes[0][1])

    for route in top_routes:
        for i in range(len(route[1])):
            if route[1][i]['route_id'] == "WALKING":
                if i != 0:
                    w_time = route[1][i]['arrival'] - route[1][i]['departure']
                    route[1][i]['departure'] = route[1][i-1]['arrival']
                    route[1][i]['arrival'] = route[1][i]['departure'] + w_time

    # Sort the top routes by latest departure time
    top_routes.sort(key=lambda x: x[0], reverse=True)
    # Return the top k routes
    return top_routes[:k]

# Example usage
#source = 'St-Sulpice VD, Castolin'
#target = 'Lausanne-Flon'
#arrival_time = parse_time('09:27:00')
#k = 5
#
#top_routes = find_top_k_trips(G, source, target, arrival_time, k)

Added edge: St-Sulpice VD, Castolin -> St-Sulpice VD, En Champagny
Added edge: St-Sulpice VD, En Champagny -> St-Sulpice VD, Ochettaz-Ormet
Added edge: St-Sulpice VD, Ochettaz-Ormet -> St-Sulpice VD, Pré-Fleuri
Added edge: St-Sulpice VD, Pré-Fleuri -> St-Sulpice VD, Parc Scient.
Added edge: St-Sulpice VD, Parc Scient. -> St-Sulpice VD, Pâqueret
Added edge: St-Sulpice VD, Pâqueret -> Ecublens VD, Champagne
Added edge: Ecublens VD, Champagne -> Ecublens VD, Blévallaire
Added edge: Ecublens VD, Blévallaire -> Ecublens VD, allée de Dorigny
Added edge: Ecublens VD, allée de Dorigny -> Lausanne, Bourdonnette
Added edge: Lausanne, Bourdonnette -> Lausanne, Malley
Added edge: Lausanne, Malley -> Lausanne, Provence
Added edge: Lausanne, Provence -> Lausanne, Montelly
Added edge: Lausanne, Montelly -> Lausanne, Vigie
Added edge: Lausanne, Vigie -> Lausanne-Flon, pl. de l'Europe
Added edge: Lausanne-Flon, pl. de l'Europe -> Lausanne-Flon
[{'from': 'St-Sulpice VD, Castolin', 'to': 'St-Sulpice VD, 

In [8]:
def concatenate_routes(data):
        concatenated_routes = []
        current_route = None
    
        for item in data[1]:
            if current_route is None:
                current_route = item
            elif item['route_id'] == current_route['route_id']:
                # Extend the current route
                current_route['to'] = item['to']
                current_route['arrival'] = item['arrival']
                current_route['to_lat'] = item['to_lat']
                current_route['to_lon'] = item['to_lon']
            else:
                # Append the finished route and start a new one
                concatenated_routes.append(current_route)
                current_route = item
        
        if current_route:
            concatenated_routes.append(current_route)
    
        return (data[0], concatenated_routes)

def concatenate_trips(trips):
    concatenated_trips = []
    for data in trips:
        concatenated_trips.append(concatenate_routes(data))
    
    return concatenated_trips 

# method that gives both data for visualistation and for delay modeling

In [9]:
def top_k_trips_for_visualisation_and_delays(G, source, target, arrival_time, k=5):
    arrival_time = parse_time(arrival_time)
    data_for_vis = find_top_k_trips(G, source, target, arrival_time, k=k)
    data_for_vis2 = copy.deepcopy(data_for_vis)  # Create a deep copy of the data
    data_for_delays = concatenate_trips(data_for_vis2)
    return data_for_delays, data_for_vis
    
# full method to find the top 5 trips with visualisation
#start_station = 'St-Sulpice VD, Venoge sud'
#end_station = "Lausanne-Flon, pl. de l'Europe"
#first_departure_time_str = '13:00:00'
#
#transform_data_delays,transform_data_for_vis =top_k_trips_for_visualisation_and_delays(G, start_station, end_station, first_departure_time_str)

# delay modeling a modifier avec le nouveau format 

In [10]:
# read the csv file
delay_clean_df = pd.read_csv('data/delay_clean_df.csv')
delay_clean_df

,stop_name,product_id,hour_cat,day_name,arrival_delay
0,"Ecublens VD, allée de Dorigny",Bus,2,Friday,223
1,"Ecublens VD, Blévallaire",Bus,2,Friday,250
2,"Ecublens VD, Champagne",Bus,2,Friday,241
3,"St-Sulpice VD, Pâqueret",Bus,2,Friday,189
4,"St-Sulpice VD, Parc Scient.",Bus,2,Friday,199
...,...,...,...,...,...
2660102,"Pully, Monts-de-Pully",Bus,2,Monday,65
2660103,"Pully, Trois-Chasseurs",Bus,2,Monday,35
2660104,"Montblesson, Centenaire",Bus,2,Monday,67
2660105,"Lausanne, Rovéréaz",Bus,2,Monday,131


In [11]:
# Step 2: Define helper functions

def to_sec(time):
    """Transform a time given in minutes format to seconds."""
    return time * 60

def exponential_distribution_proba(available_time, avg_delay):
    if available_time <= 0:
        return 0
    if avg_delay == 0:
        return 1
    proba = 1 - np.exp(-available_time / avg_delay)
    return proba

def seconds_to_rounded_hour(seconds):
    """Converts seconds to the nearest rounded down hour."""
    hours = seconds // 3600
    return hours

def get_hour_cat(hour):
    """Categorize the hour into rush time (1) or normal time (2)."""
    if hour in [7, 8, 17, 18]:
        return 1
    else:
        return 2

def get_mean_delay_for_cat(hour_cat, df):
    delay = df.loc[df['hour_cat'] == hour_cat, 'mean_delay']
    if not delay.empty:
        return delay.values[0]
    else:
        return None

def calculate_probabilities(transform_data_delays, delay_clean_df):
    results = []

    # Precompute the mean delays by hour category
    mean_delay_by_cat = delay_clean_df.groupby('hour_cat').agg({'arrival_delay': 'mean'}).reset_index()
    mean_delay_by_cat_df = mean_delay_by_cat.rename(columns={'arrival_delay': 'mean_delay'})

    for _, path in transform_data_delays:
        #print(f"Path: {path}")
        total_proba = 1.0

        for i in range(1, len(path)):
            previous_leg = path[i - 1]
            current_leg = path[i]

            previous_arrival = previous_leg['arrival']
            current_departure = current_leg['departure']
            available_time = to_sec(current_departure - previous_arrival)
            #print(f"Previous arrival: {previous_arrival}, Current departure: {current_departure}, Available time: {available_time}")

            if previous_leg['route_desc'] == 'WALKING':
                continue
            else:
                stop_name = previous_leg['to']
                hour_cat = get_hour_cat(seconds_to_rounded_hour(to_sec(previous_arrival * 60)))  # converting minutes to seconds
                product_id = previous_leg['route_desc']

                condition = (
                    (delay_clean_df['stop_name'] == stop_name) &
                    (delay_clean_df['hour_cat'] == hour_cat) &
                    (delay_clean_df['product_id'] == product_id)
                )

                if condition.any():
                    avg_delay = delay_clean_df.loc[condition, 'arrival_delay'].values[0]
                else:
                    avg_delay = get_mean_delay_for_cat(hour_cat, mean_delay_by_cat_df)

            p = exponential_distribution_proba(available_time, avg_delay)
            #print(f"Average delay: {avg_delay}, Probability: {p}")
            total_proba *= p

        results.append(total_proba)

    return results

In [12]:
def full_method(G,start_station, end_station, arrival_time,k, delay_clean_df):
    transform_data_delays, transform_data_for_vis = top_k_trips_for_visualisation_and_delays(G, start_station, end_station, arrival_time, k)
    probabilities = calculate_probabilities(transform_data_delays, delay_clean_df)
    return transform_data_delays, transform_data_for_vis, probabilities

# example usage
#start_station = 'St-Sulpice VD, Venoge sud'
#end_station = "Lausanne-Flon, pl. de l'Europe"
#arrival_time = '13:00:00'
#k = 2
#transform_data_for_vis,transform_data_delays, probabilities = full_method(G, start_station, end_station, arrival_time, k, delay_clean_df)


In [13]:
#transform_data_for_vis

In [14]:
#probabilities

# data visualisation

In [15]:
# Sample graph nodes for demonstration
G_nodes = list(G.nodes())

# Widgets for user input
departure_station = Combobox(options=G_nodes, description='From:', ensure_option=True, placeholder='Type or select a station')
arrival_station = Combobox(options=G_nodes, description='To:', ensure_option=True, placeholder='Type or select a station')
desired_arrival_hour = BoundedFloatText(min=0, max=23, value=12, step=1, description='Hour')
desired_arrival_minute = BoundedFloatText(min=0, max=59, value=0, step=1, description='Minute')
run_button = Button(description="Run query")
data_output = Output()
map_output = Output()

# Function to generate a unique color for each route ID
def generate_route_colors(paths):
    route_ids = {segment['route_id'] for path in paths for segment in path[1]}
    color_map = {route_id: f"rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})" for route_id in route_ids}
    return color_map

# Function to create map from provided data
def create_map(path, color_map):
    fig = go.Figure()

    for segment in path[1]:
        fig.add_trace(go.Scattermapbox(
            mode="lines",
            lon=[segment['from_lon'], segment['to_lon']],
            lat=[segment['from_lat'], segment['to_lat']],
            marker={'size': 10},
            hoverinfo='text',
            text=f"Route ID: {segment['route_id']}",
            line=dict(color=color_map[segment['route_id']], width=4)
        ))
    start_end_points = [{'lat': segment['from_lat'], 'lon': segment['from_lon'], 'name': segment['from']}
                        for segment in path[1]] + [{'lat': path[1][-1]['to_lat'], 'lon': path[1][-1]['to_lon'], 'name': path[1][-1]['to']}]
    fig.add_trace(go.Scattermapbox(
        lat=[p['lat'] for p in start_end_points],
        lon=[p['lon'] for p in start_end_points],
        mode='markers',
        marker=dict(size=10, color='red'),
        text=[p['name'] for p in start_end_points],
        hoverinfo='text'
    ))

    fig.update_layout(
        mapbox_style="open-street-map",
        hovermode='closest',
        width=1000,  # Set the desired width
        height=800,  # Set the desired height
        mapbox=dict(
            bearing=0,
            center=dict(
                lat=46.518, 
                lon=6.56
            ),
            pitch=0,
            zoom=10
        ),
    )

    return fig

# Function to generate HTML table from path data
def generate_table(path):
    table_html = "<table><tr><th>From</th><th>To</th><th>Departure</th><th>Arrival</th><th>Route ID</th></tr>"
    for segment in path[1]:
        table_html += f"<tr><td>{segment['from']}</td><td>{segment['to']}</td><td>{reverse_time(segment['departure'])}</td><td>{reverse_time(segment['arrival'])}</td><td>{segment['route_id']}</td></tr>"
    table_html += "</table>"
    return table_html

# Function triggered when the button is clicked
def on_button_clicked(btn):
    data_output.clear_output()
    map_output.clear_output()

    start_station = departure_station.value
    end_station = arrival_station.value
    arrival_time = f"{int(desired_arrival_hour.value):02}:{int(desired_arrival_minute.value):02}:00"

    # Replace with actual call to your method
    transform_data_for_delay, transform_data_for_vis, probabilities = full_method(G, start_station, end_station, arrival_time, 5, delay_clean_df)
    
    # Generate route colors
    color_map = generate_route_colors(transform_data_for_vis)
    
    # Create tabs for displaying data and maps
    data_output_tabs = []
    map_output_tabs = []
    
    for idx, path in enumerate(transform_data_for_vis):
        path_map_output = Output()
        
        with path_map_output:
            display(create_map(path, color_map))

        map_output_tabs.append(path_map_output)

    for idx, path in enumerate(transform_data_for_delay):
        path_data_output = Output()
        
        with path_data_output:
            display(HTML(f"<h3>Path {idx + 1} (Probability: {probabilities[idx]*100:.2f}%):</h3>"))
            display(HTML(generate_table(path)))
        
        data_output_tabs.append(path_data_output)
    
    data_tabs = Tab(children=data_output_tabs)
    map_tabs = Tab(children=map_output_tabs)
    
    
    for idx in range(len(data_output_tabs)):
        data_tabs.set_title(idx, f"Path {idx + 1}")
        map_tabs.set_title(idx, f"Map {idx + 1}")
    
    with data_output:
        display(data_tabs)
    
    with map_output:
        display(map_tabs)

run_button.on_click(on_button_clicked)

input_widgets = HBox([departure_station, arrival_station, desired_arrival_hour, desired_arrival_minute, run_button])
tab = Tab([data_output, map_output])
tab.set_title(0, 'Data')
tab.set_title(1, 'Map')

dashboard = VBox([input_widgets, tab])
display(dashboard)